In [19]:
from json import loads
import os
import networkx as ntx
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import SpectralClustering

In [2]:
data = [loads(line) for line in open('out.json', 'r')]
G = ntx.DiGraph()
papers = {}
authors = {}
for item in data:
    paper_id = item["id"]
    paper_incite = item["inCitations"]
    tmp = item["authors"]
    paper_author = []
    for t in tmp:
        try:
            AuthorId = t['ids'][0]
        except:
            AuthorId = ''
        authors[AuthorId] = t["name"]
        paper_author.append(AuthorId)
    papers[paper_id] = [paper_incite, paper_author]
#print(papers['81fe230c5f241d1dd37b9ebc05db118c1a010c3e'])
del data
G.add_nodes_from(authors.keys())
for out_id in papers:
    #print(out_id)
    if not papers[out_id][0] or not papers[out_id][1]:
        continue
    for in_id in papers[out_id][0]:
        if in_id not in papers or not papers[in_id][1]:
            continue
        for author_id in papers[out_id][1]:
            G.add_edges_from([(author_id, i) for i in papers[in_id][1]])
removal = set()
for node in G.nodes:
    if G.degree(node) == 0:
        removal.add(node)
G.remove_nodes_from(removal)

In [15]:
print(G.number_of_edges()
print(G.number_of_nodes())

22167
9464


In [17]:
## Clustering
adj_mat = ntx.to_numpy_matrix(G)
sc = SpectralClustering(10, affinity='precomputed', n_init=100)
sc.fit(adj_mat)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/manifold/_spectral_embedding.py:212: UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
  adjacency = check_symmetric(adjacency)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


SpectralClustering(affinity='precomputed', assign_labels='kmeans', coef0=1,
                   degree=3, eigen_solver=None, eigen_tol=0.0, gamma=1.0,
                   kernel_params=None, n_clusters=10, n_components=None,
                   n_init=100, n_jobs=None, n_neighbors=10, random_state=None)

In [34]:
## Save nodes
import csv
with open('Authors.csv', 'w') as f:
    for key in G.nodes:
        f.write("%s,%s\n"%(key,authors[key]))
with open('Edges.csv', 'w') as f:
    for key in G.edges:
        f.write("%s,%s\n"%(key[0],key[1]))